In [1]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections
from importlib import reload
import pandas as pd
import sklearn

In [2]:
import xgboost as xgb

import sklearn.ensemble
import sklearn.svm
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [3]:
# Consider the training dataset
# Split the data up in n subsets, based on `search_id` to prevent occurence of a `search_id` in multiple sets
# Resample each subset, now based on classes: select 1/3 booking, 1/3 click (but no booking), 1/3 none
#     The resampling is used the preserve the class sizes
# Use crossvalidation on the n sets to select hyperparams
# Finally train model on full training dataset and make a prediction of the (unseen) test dataset

In [56]:
import util.data
import util.model

In [5]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=2*1000)
data_test = pd.read_csv('data/test_set_VU_DM_clean.csv', sep=';', nrows=1000)
# scores = pd.read_csv('data/scores_train.csv', sep=';')
data.drop(columns=['position'], inplace=True)

In [6]:
for k in data.columns:
    if data[k].isna().sum() > 0:
        print('rm ', data[k].isna().sum() / data.shape[0], k)
        data.drop(columns=[k], inplace=True)

rm  0.9815 comp1_rate
rm  0.98 comp1_inv
rm  0.984 comp1_rate_percent_diff
rm  0.59 comp2_rate
rm  0.5705 comp2_inv
rm  0.8935 comp2_rate_percent_diff
rm  0.6495 comp3_rate
rm  0.6335 comp3_inv
rm  0.9185 comp3_rate_percent_diff
rm  0.94 comp4_rate
rm  0.9315 comp4_inv
rm  0.9785 comp4_rate_percent_diff
rm  0.5105 comp5_rate
rm  0.4855 comp5_inv
rm  0.817 comp5_rate_percent_diff
rm  0.9235 comp6_rate
rm  0.9175 comp6_inv
rm  0.9655 comp6_rate_percent_diff
rm  0.9075 comp7_rate
rm  0.8915 comp7_inv
rm  0.96 comp7_rate_percent_diff
rm  0.5755 comp8_rate
rm  0.556 comp8_inv
rm  0.876 comp8_rate_percent_diff


In [57]:
y_labels = [ k for k in data.columns if k not in data_test.columns ]
y_labels

['click_bool', 'booking_bool', 'score']

In [58]:
def split_xy(data: pd.DataFrame, y_label='score'):
    return data.drop(columns=y_labels).values, data[y_label].values

In [59]:
reload(util.data)
folds = util.data.cv_folds_for_sklearn(data, n_cv_folds=5, resampling_ratio=0)
x, y = split_xy(data)

In [79]:
seed = 123
models = {
#           'Logit': sklearn.linear_model.LogisticRegression(solver='liblinear',
#                                                            multi_class='ovr'),
# #           'SGD': sklearn.linear_model.SGDClassifier(loss="hinge", penalty="l2", max_iter=1000, tol=1e-3),
#           'SVM': sklearn.svm.SVR(kernel='linear'),
          'Decision Tree':  sklearn.tree.DecisionTreeClassifier(),
#           'KNN 10': sklearn.neighbors.KNeighborsClassifier(n_neighbors=10),
          'Ensemble Random Forest': sklearn.ensemble.RandomForestClassifier(n_estimators=100),
#             'Gradient Boost': sklearn.ensemble.GradientBoostingRegressor(loss='ls', learning_rate=0.1, 
#                             n_estimators=100, subsample=1.0, criterion='friedman_mse', 
#                             max_depth=3,random_state=seed, alpha=0.9, tol=0.0001)    
         'AdaBoost': sklearn.ensemble.AdaBoostRegressor(),
          'Gradient Boost': xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.01, 
                          max_depth = 4, alpha = 10, n_estimators = 500, n_jobs=2, seed=123)
         }

In [80]:
reload(util.model)
n_folds_used = 5
results = {}
for k,m in models.items():
    print(k)
    util.model.cross_validation(m, x, y, folds[:n_folds_used], k, results)

Decision Tree
	 scoring: explained_variance
scores per fold  [-0.7823, -0.1613, 0.4868, -0.3942, -1.7196]
  mean score     -0.5141237447359895
  standard dev.  0.7301631882599674
Ensemble Random Forest
	 scoring: explained_variance
scores per fold  [-0.3158, -0.896, -0.119, -0.3366, 0.3333]
  mean score     -0.2668421336788683
  standard dev.  0.39627909860175575
AdaBoost
	 scoring: explained_variance
scores per fold  [-0.2719, 0.299, 0.6323, 0.1855, 0.9336]
  mean score     0.35571003690180514
  standard dev.  0.4090201366793554
Gradient Boost
	 scoring: explained_variance
scores per fold  [0.1445, 0.2856, 0.3264, 0.1966, 0.4465]
  mean score     0.2799185131606986
  standard dev.  0.10508558687659222


In [81]:
util.model.scores_table(results)

Model & Mean & Std. dev. \\ 
\hline
Decision Tree & -0.5141 & 0.7302\\
Ensemble Random Forest & -0.2668 & 0.3963\\
AdaBoost & 0.3557 & 0.4090\\
Gradient Boost & 0.2799 & 0.1051\\

best score: AdaBoost, with mean: 0.3557


In [78]:
y1 = y[folds[0][0]]
y1[y1 == 5].shape, y1[y1 == 0].shape, y1[y1 == 1].shape

((22,), (22,), (22,))

In [15]:
# some tests
reload(util.data)

n_cv_folds = 5
ids = sklearn.utils.shuffle(data.srch_id.unique(), random_state=123)
ids_per_fold  = np.array_split(ids, n_cv_folds)
data_splits = util.data.split_data_based_on_ids(data, ids_per_fold)
sum_ = sum([split.shape[0] for split in data_splits])
assert sum_ == data.shape[0], (sum_, data.shape[0])

for i in range(min(2,len(data_splits))):
    for j in range(min(4, len(data_splits))):
        # check index
        if i != j:
            for idx in data_splits[i].index:
                assert idx not in data_splits[j].index

        # check attr srch_id
        if i != j:
            for srch_id in data_splits[i].srch_id:
                assert data_splits[j].query('srch_id == @srch_id').srch_id.size == 0

In [17]:
bco_splits = [ util.data.split_bookings_clicks_others(data) for data in data_splits ]

for i in range(len(bco_splits)):
    for j in range(len(bco_splits)):
        if i != j:
            for srch_id in bco_splits[i][0].srch_id:
                assert bco_splits[j][0].query('srch_id == @srch_id').shape[0] == 0
                assert bco_splits[j][1].query('srch_id == @srch_id').shape[0] == 0
                assert bco_splits[j][2].query('srch_id == @srch_id').shape[0] == 0

In [20]:
# e.g. for every cv split i
i = 0
bco_split = bco_splits[i]
bookings, clicks, others = bco_split
size_per_sample = 2
assert len(util.data.sample([bookings, clicks, others], size_per_sample)) == 3 * size_per_sample